In [50]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score 
import nltk 


In [51]:
nltk.download('stopwords')
from nltk.corpus import stopwords
data = pd.read_csv("D:/SEM 6/MACHINE LEARNING/Project/twitter_training.csv")
data


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nihar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [52]:
data.columns = ["ID", "Category", "Sentiment", "Text"]

In [53]:
data = data.drop_duplicates() 
# import re
# from nltk.corpus import stopwords

# # Function to clean tweets
# def clean_tweet(tweet):
#     stop_words = set(stopwords.words('english'))
#     tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)  # Remove URLs
#     tweet = re.sub(r'@\w+|#', '', tweet)  # Remove mentions and hashtags
#     tweet = re.sub(r'\W', ' ', tweet)  # Remove non-alphanumeric characters
#     tweet = tweet.lower()  # Convert to lowercase
#     tokens = nltk.word_tokenize(tweet)
#     cleaned_tokens = [word for word in tokens if word not in stop_words]
#     return ' '.join(cleaned_tokens)


In [54]:
data = data.dropna(subset=["Text"])

In [55]:
label_encoder = LabelEncoder()
data["Sentiment"] = label_encoder.fit_transform(data["Sentiment"])

In [56]:
X = data["Text"]  # Text data (tweets)
y = data["Sentiment"]  # Sentiment labels

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [63]:
tfidf = TfidfVectorizer(max_features=5000, stop_words="english")
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [69]:
import pickle

In [71]:
# Save the vectorizer as vectorizer.pkl
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

In [64]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [65]:
y_pred = log_reg.predict(X_test_tfidf)
y_pred

array([0, 2, 2, ..., 1, 0, 1])

In [66]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

In [67]:
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", report)

Model Accuracy: 69.23%

Classification Report:
               precision    recall  f1-score   support

  Irrelevant       0.69      0.52      0.60      2507
    Negative       0.69      0.81      0.75      4340
     Neutral       0.70      0.63      0.66      3542
    Positive       0.69      0.72      0.71      3942

    accuracy                           0.69     14331
   macro avg       0.69      0.67      0.68     14331
weighted avg       0.69      0.69      0.69     14331



In [72]:
# Save the trained model
with open("model.pkl", "wb") as f:
    pickle.dump(log_reg, f)